In [29]:
!pip install transformers

In [30]:
import json
import os
import json

In [31]:
if not os.path.exists('responses'):
    os.mkdir('responses')

In [38]:
def open_file(filepath):
    with open(filepath, 'r', encoding = 'utf-8') as infile:
        return infile.read()

In [39]:
def save_file(filepath, content):
    with open(filepath, 'w', encoding = 'utf-8') as outfile:
        outfile.write(content)

In [51]:
task = open_file('task.txt')
base_solver = open_file('prompt1.txt')
chatbot_prompt = open_file('sysprompt.txt')

In [56]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "openai-community/gpt2"  # You can choose other models from Hugging Face
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to generate text using Hugging Face model
def generate_text(prompt, temperature=0.7, max_length=512, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to handle long texts by splitting into manageable chunks
def generate_text_in_chunks(prompt, temperature=0.7, chunk_size=512, max_new_tokens=50):
    # Tokenize the prompt and get input_ids
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=chunk_size)
    input_ids = inputs["input_ids"][0]

    # Split the input_ids into chunks of chunk_size tokens
    input_chunks = [input_ids[i:i + chunk_size] for i in range(0, len(input_ids), chunk_size)]
    generated_text = ""

    for chunk in input_chunks:
        # Convert chunk back to text
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        # Generate text for each chunk
        generated_chunk_text = generate_text(chunk_text, temperature=temperature, max_length=chunk_size, max_new_tokens=max_new_tokens)
        generated_text += generated_chunk_text

    return generated_text

# Function to simulate chatgpt
def chatgpt(nconversation, chatbot_prompt, solver, temperature=0.7):
    nconversation.append({"role": "user", "content": solver})
    messages_input = nconversation.copy()
    prompty = [{"role": "system", "content": chatbot_prompt}]
    messages_input.insert(0, prompty[0])

    prompt = "\n".join([message['content'] for message in messages_input])
    chat_response = generate_text(prompt, temperature=temperature)
    nconversation.append({"role": "assistant", "content": chat_response})

    return chat_response

# Example usage
jsonl_file = "responses/schema.jsonl"
num_loops = 4
conversation = []
nconversation = list(conversation)  # Initialize conversation list

for i in range(num_loops):
    prob1 = chatgpt(nconversation, chatbot_prompt, note)
    solver = base_solver.replace("<<NOTES>>", prob1)
    response = chatgpt(nconversation, chatbot_prompt, solver)

    json_obj = {
        "messages": [
            {"role": "system", "content": chatbot_prompt},
            {"role": "user", "content": prob1},
            {"role": "assistant", "content": response}
        ]
    }

    with open(jsonl_file, 'a') as f:
        f.write(json.dumps(json_obj) + '\n\n')

    print(f"Saved example {i+1} to {jsonl_file}")

    nconversation.clear()
    


Saved example 1 to responses/schema.jsonl
Saved example 2 to responses/schema.jsonl
Saved example 3 to responses/schema.jsonl
Saved example 4 to responses/schema.jsonl


In [57]:
print(solver)

CLINICAL NOTE:

You are a Healthcare Official Chatbot involved in the Protected Health Information that list out different situations of patients social history/sdoh.
Your task is to Use the structure of the data below as an example, create 2 new data examples of {NOTES}, putting each example on a new line in a table that I can easily copy. you must maintain the structure of the examples below. Every example must have the line ”messages”:
[{”role”: ”system”, ”content”: “You are a Healthcare Official Chatbot involved in the Protected Health Information that list out different situations of patients social history/sdoh.” Keep Certain important elements like years, months, names, states, citys, countrys and so on should be de-identified if not already. Create new examples for the user content.

Here are some examples of how the notes should look when generated:

{”messages”:
[{”role”: ”system”, ”content”: “You are a Healthcare Official Chatbot involved in the Protected Health Information 